# Library importing

In [1]:
import pandas as pd
import joblib
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

# data pre-proccessing

## data reading

In [2]:
chunks = pd.read_csv("/kaggle/input/data.csv", chunksize=5_000_000)
final_data = pd.concat(chunks, ignore_index=True)

In [3]:
del chunks

In [4]:
preprocessor=joblib.load("/kaggle/input/pre-proccess.pkl")

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info 

## Data spliting

In [5]:
y=final_data['sales']
x=final_data.drop('sales',axis = 1)

# Model

In [6]:
del final_data
import gc
gc.collect()

0

In [7]:
categorical_cols = ['store_id', 'event_name_1', 'event_name_2', 'event_type_1', 'event_type_2', 'state_id', 'item_category', 'item_subcategory']
numeric_cols = ["wm_yr_wk", "wday", "snap","year", "month", "day", "sell_price","lag_1",'price_flag','lag_7','snap_weekend','wday_x_snap','is_weekend']
for col in categorical_cols:
    if x[col].dtype == "object":
        x[col] = x[col].astype("category")
for col in numeric_cols:
    x[col] = pd.to_numeric(x[col], downcast="float")
del categorical_cols
del numeric_cols
gc.collect()

0

In [8]:
del col

In [9]:
gc.collect()

0

In [10]:
x=preprocessor.fit_transform(x)

In [34]:
model = XGBRegressor(
    objective='reg:tweedie',
    tweedie_variance_power=1.35,
    booster='gbtree',
    device='cuda',
    tree_method='hist',
    random_state=42,
    eval_metric='mae',
    learning_rate=0.04,
    max_depth=10,
    min_child_weight=10,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=6,
    reg_alpha=2,
    n_estimators=3000,
    early_stopping_rounds=40,
    gamma=0.1
)

In [12]:
x_temp , x_test , y_temp , y_test = train_test_split(x , y , test_size = 0.20 , random_state = 42)
del x,y
gc.collect()
x_train , x_val , y_train , y_val = train_test_split(x_temp , y_temp , test_size = 0.15 , random_state = 42)
del x_temp,y_temp
gc.collect()

0

In [13]:
gc.collect()

0

In [27]:
from sklearn.metrics import mean_absolute_error, r2_score

train_mae = mean_absolute_error(y_train, model.predict(x_train))
test_mae  = mean_absolute_error(y_test,  model.predict(x_test))
train_r2  = r2_score(y_train, model.predict(x_train))
test_r2   = r2_score(y_test,  model.predict(x_test))

print(f"MAE train: {train_mae:.4f}, test: {test_mae:.4f}")
print(f"R2  train: {train_r2:.4f}, test: {test_r2:.4f}")

MAE train: 0.7628, test: 0.7910
R2  train: 0.7906, test: 0.7325


In [35]:
model.fit(x_train,y_train,eval_set = [(x_val , y_val)])

[0]	validation_0-mae:1.52798
[1]	validation_0-mae:1.49493
[2]	validation_0-mae:1.45986
[3]	validation_0-mae:1.43040
[4]	validation_0-mae:1.40212
[5]	validation_0-mae:1.37576
[6]	validation_0-mae:1.35125
[7]	validation_0-mae:1.32915
[8]	validation_0-mae:1.30705
[9]	validation_0-mae:1.28620
[10]	validation_0-mae:1.26654
[11]	validation_0-mae:1.24807
[12]	validation_0-mae:1.23074
[13]	validation_0-mae:1.21419
[14]	validation_0-mae:1.19849
[15]	validation_0-mae:1.18353
[16]	validation_0-mae:1.16938
[17]	validation_0-mae:1.15971
[18]	validation_0-mae:1.14670
[19]	validation_0-mae:1.13434
[20]	validation_0-mae:1.12248
[21]	validation_0-mae:1.11114
[22]	validation_0-mae:1.10137
[23]	validation_0-mae:1.09096
[24]	validation_0-mae:1.08102
[25]	validation_0-mae:1.07153
[26]	validation_0-mae:1.06340
[27]	validation_0-mae:1.05458
[28]	validation_0-mae:1.04611
[29]	validation_0-mae:1.03805
[30]	validation_0-mae:1.03104
[31]	validation_0-mae:1.02352
[32]	validation_0-mae:1.01625
[33]	validation_0-ma

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device='cuda', early_stopping_rounds=40,
             enable_categorical=False, eval_metric='mae', feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.04, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=10, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=3000, n_jobs=None,
             num_parallel_tree=None, objective='reg:tweedie', ...)

In [36]:
model.score(x_train,y_train) , model.score(x_test,y_test)

(0.7695187321330575, 0.729663725173127)

In [41]:
import numpy as np
y_predict=model.predict(x_test)
y_pred_rounded = np.round(y_predict).astype(int)
y_test_rounded = np.round(y_test).astype(int)
ct=0
for i in range (0,25):
    print(y_pred_rounded[i], y_test_rounded.iloc[i])
    if(y_pred_rounded[i]==y_test_rounded.iloc[i]):
        ct+=1

6 2
1 1
1 1
1 1
0 0
0 0
1 0
0 0
3 2
0 0


0.7